In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install -U -q tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 25.1 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np


In [ ]:
%pip install image-dataset-loader

In [ ]:
!pip install scikit-image



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np


In [ ]:
def crop_face(img):
  detector = dlib.get_frontal_face_detector()
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  gray = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

  gray = np.array(gray, dtype=np.uint8)
  image_height, image_width = gray.shape

  rects = detector(gray)
  #print(len(rects))
  if len(rects)<1:
    if 'CROP_PARAM_LAST' not in locals():
      CROP_PARAM_LAST = [114, 293, 154, 333]
    crop_img = img[max(0, CROP_PARAM_LAST[0]):min(CROP_PARAM_LAST[1], image_height), max(0, CROP_PARAM_LAST[2]):min(CROP_PARAM_LAST[3], image_width)]
    crop_img_gray = gray[max(0, CROP_PARAM_LAST[0]):min(CROP_PARAM_LAST[1], image_height), max(0, CROP_PARAM_LAST[2]):min(CROP_PARAM_LAST[3], image_width)]
    return crop_img, crop_img_gray

  for det in rects:
    CROP_PARAM_LAST = [max(0, det.top()), min(det.bottom(), image_height), max(0, det.left()), min(det.right(), image_width)]
    print(CROP_PARAM_LAST)
    crop_img = img[max(0, det.top()):min(det.bottom(), image_height), max(0, det.left()):min(det.right(), image_width)]
    crop_img_gray = gray[max(0, det.top()):min(det.bottom(), image_height), max(0, det.left()):min(det.right(), image_width)]
  return crop_img, crop_img_gray

In [ ]:
"""import os
from PIL import Image
from google.colab.patches import cv2_imshow  # Required to display images in Colab

# Path to the folder containing the images
folder_path = "/content/drive/MyDrive/Deepfakes/Data/Task_1/evaluation/real"
out_folder = "/content/drive/MyDrive/Deepfakes/Data/Task_1/faces/evaluation/real/"

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an image
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Load the image
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)
        # Apply transformation (e.g. flip horizontally)
        cropped, _ = crop_face(image)
        
        # Save the transformed image
        new_filename =  os.path.join(out_folder, filename)
        print(new_filename)
        cv2.imwrite(new_filename, cropped)

"""


In [ ]:
import cv2
import numpy as np
import dlib
import os


# Define paths to real and fake images

basepath="drive/MyDrive/Deepfakes/Data/Task_1/faces"
train_data_dir = "/content/drive/MyDrive/Deepfakes/Data/Task_1/faces/development"
validation_data_dir = "/content/drive/MyDrive/Deepfakes/Data/Task_1/evaluation"
real_path = train_data_dir +"/real"
fake_path = train_data_dir +"/fake"

img_width, img_height = 436, 500



detector = dlib.get_frontal_face_detector()

predictor = dlib.shape_predictor("/content/drive/MyDrive/Deepfakes/models/shape_predictor_68_face_landmarks.dat")

# Define a function to extract features from an image
def extract_features(image):
    #gray = cv2.convertScaleAbs(image, alpha=(255.0 / 65535.0))
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Normalize the pixel values to the range [0, 255]
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

    # Convert the pixel values from float32 to uint8 data type
    gray = np.array(img, dtype=np.uint8)

    print(gray.dtype)
    features = []

    # Detect faces in the grayscale image
    faces = detector(gray)

    if len(faces) == 0:
        print('No faces detected')
        return None
    elif len(faces) > 1:
        print('Multiple faces detected')
        return None
    else:
        print('One face detected')
        # Use the shape predictor model to get landmarks
        landmarks = predictor(gray, faces[0])
        # Convert landmarks to a numpy array
        landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])

        for (x, y) in landmarks:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        #cv2.imshow('Landmarks', gray)

        # Compute handcrafted features from the facial landmarks
        landmarks = np.squeeze(landmarks)
        eye_distances = [np.linalg.norm(landmarks[36] - landmarks[45]), np.linalg.norm(landmarks[42] - landmarks[47])]
        nose_width = np.linalg.norm(landmarks[31] - landmarks[35])
        mouth_width = np.linalg.norm(landmarks[48] - landmarks[54])
        features.extend([eye_distances[0], eye_distances[1], nose_width, mouth_width])
        #do fft for gray img
        fft = np.fft.fft2(gray, s=[100,100], norm="ortho").reshape(1,-1)
        fft = np.absolute(fft).tolist()[0] #take magnitude and convert to list

        return features# + fft

# Load the dataset and extract features and labels


# Define a function to extract features from an image
def extract_features2(image):
    #gray = cv2.convertScaleAbs(image, alpha=(255.0 / 65535.0))
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Normalize the pixel values to the range [0, 255]
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

    # Convert the pixel values from float32 to uint8 data type
    gray = np.array(img, dtype=np.uint8)
    features = []

    # Detect faces in the grayscale image
    faces = detector(gray)

    if len(faces) == 0:
        print('No faces detected')
        return None
    elif len(faces) > 1:
        print('Multiple faces detected')
        return None
    else:
        print('One face detected')
        # Use the shape predictor model to get landmarks
        landmarks = predictor(gray, faces[0])
        # Convert landmarks to a numpy array
        landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])

        for (x, y) in landmarks:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        #cv2.imshow('Landmarks', gray)

        # Calculate the distances between the left and right facial landmarks
        landmarks = np.squeeze(landmarks)

        eye_distances = [np.linalg.norm(landmarks[36] - landmarks[45]), np.linalg.norm(landmarks[42] - landmarks[47])]
        nose_width = np.linalg.norm(landmarks[31] - landmarks[35])
        mouth_width = np.linalg.norm(landmarks[48] - landmarks[54])
        features.extend([eye_distances[0], eye_distances[1], nose_width, mouth_width])

        
        left_eye_dist = np.linalg.norm(landmarks[36] - landmarks[39]) + np.linalg.norm(landmarks[37] - landmarks[38]) + np.linalg.norm(landmarks[40] - landmarks[41])
        right_eye_dist = np.linalg.norm(landmarks[42] - landmarks[45]) + np.linalg.norm(landmarks[43] - landmarks[44]) + np.linalg.norm(landmarks[46] - landmarks[47])
        nose_dist = np.linalg.norm(landmarks[31] - landmarks[35]) + np.linalg.norm(landmarks[32] - landmarks[34])
        mouth_dist = np.linalg.norm(landmarks[48] - landmarks[54]) + np.linalg.norm(landmarks[49] - landmarks[53]) + np.linalg.norm(landmarks[50] - landmarks[52]) + np.linalg.norm(landmarks[59] - landmarks[55]) + np.linalg.norm(landmarks[60] - landmarks[56]) + np.linalg.norm(landmarks[61] - landmarks[57])
        

        left_mouth_dist = np.linalg.norm(landmarks[48] - landmarks[54]) + np.linalg.norm(landmarks[49] - landmarks[53]) + np.linalg.norm(landmarks[50] - landmarks[52])
        right_mouth_dist = np.linalg.norm(landmarks[55] - landmarks[59]) + np.linalg.norm(landmarks[56] - landmarks[58]) + np.linalg.norm(landmarks[57] - landmarks[61])
        features.extend([right_mouth_dist, left_mouth_dist])

        # Calculate the symmetry ratio for each feature
        eye_symmetry = left_eye_dist / right_eye_dist
        mouth_symmetry = left_mouth_dist / right_mouth_dist

        # Add the symmetry ratios to the features list
        features.extend([mouth_symmetry, eye_symmetry]) #, nose_symmetry, mouth_symmetry])

        # Calculate the distances between facial landmarks that correspond to specific expressions
        smile_dist = np.linalg.norm(landmarks[48] - landmarks[54]) + np.linalg.norm(landmarks[49] - landmarks[53]) + np.linalg.norm(landmarks[50] - landmarks[52])

        # Add the expression distances to the features list
        features.extend([smile_dist])

        left_eye = landmarks[36:42]
        right_eye = landmarks[42:48]

        # Compute the center of each eye region
        left_eye_center = np.mean(left_eye, axis=0).astype(int)
        right_eye_center = np.mean(right_eye, axis=0).astype(int)

        # Compute the horizontal and vertical distance between the eye centers
        dx = right_eye_center[0] - left_eye_center[0]
        dy = right_eye_center[1] - left_eye_center[1]

        # Compute the angle of the line between the eye centers
        #angle = np.arctan2(dy, dx) * 180 / np.pi ###########################################################
        #features.append(angle)

        #do fft for gray img
        fft = np.fft.fft2(gray, s=[100,100], norm="ortho").reshape(1,-1)
        fft = np.absolute(fft).tolist()[0] #take magnitude and convert to list

        return features  #+ fft



train_data_generator =ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    brightness_range=[0.5, 1.5],
    channel_shift_range=50,
    zoom_range=0.2)

train_generator = train_data_generator.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=1,
        class_mode='binary',
        color_mode='rgb')

# Iterate over the training images and extract features
train_features = []
train_labels = []
i = 0


num_images = len(train_generator.filenames)  # get total number of images in the dataset
counter = 0  # initialize a counter

for image, label in train_generator:
    image = image[0]  # handle 1 element batch
    label = int(label[0])
    features = extract_features2(image)
    if features:
        train_features.append(features)
        temp = [0, 0]
        temp[label] = 1
        train_labels.append(temp)
    counter += 1  # increment the counter
    if counter == num_images:  # break out of the loop when all images have been processed
        break

train_features = np.array(train_features)
train_labels = np.array(train_labels)

# Do the same for the validation set
validation_data_generator = ImageDataGenerator(rescale=1./255)

validation_generator = validation_data_generator.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=1,
        class_mode='binary')

counter = 0  # initialize a counter
num_images = len(validation_generator.filenames)  # get total number of images in the dataset

validation_features = []
validation_labels = []
for image, label in validation_generator:
    image = image[0]  # handle 1 element batch
    label = int(label[0])
    features = extract_features2(image)
    if features:
        validation_features.append(features)
        temp = [0, 0]
        temp[label] = 1
        validation_labels.append(temp)
    counter += 1  # increment the counter
    if counter == num_images:  # break out of the loop when all images have been processed
        break

validation_features = np.array(validation_features)
validation_labels = np.array(validation_labels)



Found 757 images belonging to 2 classes.
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
No faces detected
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
No faces detected
One face detected
One face detected
One face detected
One face detected
One face detected
No faces detected
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
One face detected
No faces detected
One face detected
One face detected
No faces detected
One face detected
One face detected
One face detected
One face detected
One face detected
No faces detected
One face detected
One face detected
One face detected
No fa

In [ ]:
tffilename = "train_features_8_me_symmetry.npy"
tlfilename = "train_labels_8_me_symmetry.npy"
vffilename = "validation_features_8_me_symmetry.npy"
vlfilename = "validation_labels_8_me_symmetry.npy"

np.save(tffilename, train_features)
np.save(tlfilename, train_labels)
np.save(vffilename, validation_features)
np.save(vlfilename, validation_labels)
"helloWorld"

'helloWorld'

In [ ]:
import tensorflow as tf
import numpy as np

handcrafted_features_train = np.load('/content/' + tffilename ,allow_pickle=True)#.reshape((-1, 10004))
train_labels = np.load('/content/' + tlfilename)#.reshape((-1,2))
handcrafted_features_val = np.load('/content/' + vffilename ,allow_pickle=True)#.reshape((-1, 10004))
val_labels = np.load('/content/' + vlfilename)#.reshape((-1,2))

print(handcrafted_features_train.shape)
print(handcrafted_features_val.shape)

print(train_labels.shape)
print(val_labels.shape)


# Define the data generators for train and validation sets
train_dataset = tf.data.Dataset.from_tensor_slices((handcrafted_features_train, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((handcrafted_features_val, val_labels))

def schedule(epoch, lr):
    if epoch < 200:
        return 0.00001
    elif epoch < 300:
        return 0.000001
    else:
        return 0.000001


input_shape = (9)
from keras.models import Sequential
model = Sequential([
    tf.keras.Input(shape=input_shape, name="input_layer"),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(800, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.000005),
              metrics=['accuracy'])
history = model.fit(train_dataset.batch(16),
                    epochs=200,
                    validation_data=val_dataset.batch(16))

# Calculate the loss and accuracy on the validation set
val_loss, val_acc = model.evaluate(val_dataset.batch(16))
print('Validation loss:', val_loss)
print('Validation accuracy:', val_acc)

(672, 9)
(153, 9)
(672, 2)
(153, 2)
Epoch 1/200
42/42 [==============================] - 2s 7ms/step - loss: 4.3564 - accuracy: 0.4732 - val_loss: 1.2393 - val_accuracy: 0.4575
Epoch 2/200
42/42 [==============================] - 0s 4ms/step - loss: 3.7199 - accuracy: 0.4911 - val_loss: 1.0983 - val_accuracy: 0.4575
Epoch 3/200
42/42 [==============================] - 0s 4ms/step - loss: 3.1170 - accuracy: 0.4926 - val_loss: 0.9729 - val_accuracy: 0.4575
Epoch 4/200
42/42 [==============================] - 0s 4ms/step - loss: 2.4902 - accuracy: 0.4792 - val_loss: 0.8774 - val_accuracy: 0.4575
Epoch 5/200
42/42 [==============================] - 0s 4ms/step - loss: 1.9594 - accuracy: 0.4836 - val_loss: 0.8062 - val_accuracy: 0.4575
Epoch 6/200
42/42 [==============================] - 0s 4ms/step - loss: 1.5696 - accuracy: 0.4807 - val_loss: 0.7545 - val_accuracy: 0.4575
Epoch 7/200
42/42 [==============================] - 0s 4ms/step - loss: 1.2029 - accuracy: 0.4940 - val_loss: 0.7226 

In [ ]:
# evaluate the model on test data
loss, accuracy = model.evaluate(handcrafted_features_val, val_labels, batch_size=16)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

10/10 [==============================] - 0s 3ms/step - loss: 0.6865 - accuracy: 0.6340
Test loss: 0.68650221824646
Test accuracy: 0.6339869499206543
